In [1]:
#selector de columnas a desmantelar
def SSRC(layout):
    best_ev = 0; s_o=-1
    for i in range(len(layout.stacks)):
        if layout.is_sorted_stack(i) and Greedy.capacity(layout,i)<3*len(layout.stacks[i]): continue
        
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        if layout.is_sorted_stack(i):
            ev = 100000 - 1000*len(layout.stacks[i]) - prom
        else:
            ev = 100000 - 1000*len(layout.stacks[i]) + prom
        

        if ev > best_ev:
            best_ev = ev
            s_o = i
    return s_o

In [2]:
def SDpp(layout, s_o):
    #Se calculan los ranks
    rank = Greedy.get_ranks(layout.stacks[s_o])
    
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = 0

    #eliminar de rank elementos bien colocados
    while len(layout.stacks[s_o]) > 0: #fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d, xg = Layout.select_destination_stack (layout, s_o, max_pos=rank.index(top)+1+slack)#, rank=rank)
        if verbose: print(layout.stacks)
        pos = layout.H - len(layout.stacks[s_d])
        
        if rank.index(top)+1 < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            #print()
            if verbose: 
                for s in layout.stacks: print(s)
                print("fill_stack")
                print(s_d,(pos - slack) - rank.index(top)-1,s_o, rank)
            success = Greedy.fill_stack(layout, s_d, (pos - slack) - (rank.index(top)+1), s_o, rank)
            if not success: return False, rank
            #layout.stacks[s_o].append(c)
        capac -= 1
        c = layout.move(s_o,s_d)
        if xg: rank.remove(c)
        if Layout.reachable_height(layout,s_o) >= layout.H: 
            for item in layout.stacks[s_o]: rank.remove(item)
            return True, rank
        
     #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
    for item in layout.stacks[s_o]: rank.remove(item)
    return True, rank

SFpp
---

Debería realizar un SF, priorizando los elementos (`rank`) que deberían ir en el stack original `s_d`.

````
while rank is not empty:
    s_o, s_dd = SF() # se obtienen los stacks asociados a un SF
    if s_d == s_dd: 
       if gvalue(s_dd) >= rank[0]: move(s_o, s_d)  #si stack coincide con original y el contenedor
                                                   #es mayor o igual al primero del ranking, se realiza el movimiento
       else: force_move(rank.pop(), s_d) #si no, se fuerza a mover siguiente contenedor del ranking (o mayor) a s_d
    else:
        move(s_o, s_dd)        #en caso de que SD no interfiera con el stack original, se realiza el movimiento y
        rank.pop(gvalue(s_dd))         #contenedor de elimina de stack
````



In [3]:
import sys
def SFpp(layout, s_d, rank):
    ub = Layout.gvalue(layout.stacks[s_d])
    conts = set()

    for r in rank:
        if r>ub: continue
        if r in conts: 
            conts.remove(r)
            continue
        
        while True:
            c = -1
            s, _ = Greedy.SF_move_(layout)
            s_o = None; s_dd = None
            if s != False: s_o, s_dd = s

            if s != False and s_d == s_dd and layout.stacks[s_o][-1] >= r: 
                c = layout.move(s_o,s_d)
                ub = c
            elif s==False or s_d == s_dd: #force move               
                if verbose==True:
                    if s!=False: print("rank:",r,"  c:",layout.stacks[s_o][-1], "s_o, s_d:",s_o,s_d)
                    else: print("rank:",r)
                    print(layout.stacks)
                ret = Greedy.search_highest(layout, r, ub, s_d) 
                if ret == None: ub = r-1; break
                (s, pos) = ret

                c, state = Greedy.force_move(layout, s, pos, s_d) 
                if state == "failed" and ub == r: print ("NO DEBERIA"); break # NO DEBERIA OCURRIR
                ub = c
            else:
                c = layout.move(s_o, s_dd)

            if len(layout.stacks[s_d]) == layout.H: return True
            #if len(layout.stacks[s_d]) == layout.H-1 and layout.full_stacks >= len(layout.stacks)-3: return True
            
            if c==r: break
            conts.add(c)
    return True
        

In [4]:
def solve(layout, limit=1000):    
        Greedy.lazy_greedy(layout)

        while layout.unsorted_stacks>0 and layout.steps<limit:
            seleccionada = SSRC(layout)
        
            if verbose:
                for s in layout.stacks: print(s)
                print("\n")


            if verbose: 
                print("selected:",layout.stacks[seleccionada])
                #print("nb_unsorted_elements",nb_unsorted_elements)
                #print("ranks:",rank)

            #Se retiran los elementos de la columna
            success, rank = SDpp(layout, seleccionada)
            if not success: return 1000
            #gr.SD_move(layout, stack=seleccionada)

            if verbose:
                for s in layout.stacks: print(s)
                print("\n")


            #Según yo el último parámetro implica que el método tratará de llenar el stack, sin embargo si pongo 100 se cae?
            success = SFpp(layout,seleccionada, rank)#nb_unsorted_elements)
            if not success: return 1000

            Greedy.lazy_greedy(layout)


        #print("SELECCIONADA:", seleccionada)
        #print(len(layout.moves),"\t",layout.unsorted_stacks)
        return layout.steps
        #print()

In [5]:
import copy
def BSG(layout, w=4):
    best = 200
    S = [layout]
    while len(S)>0:
        C=[]
        for lay in S:
            C = C + generate_candidates(lay)
        SS = []
        #print(len(C))
        for clay in C:
            steps_old = clay.steps
            lb_steps = steps_old + (clay.total_elements-sum(clay.sorted_elements))
            gclay=copy.deepcopy(clay)
            steps = solve(gclay, limit=best+20)
            if steps < best and gclay.unsorted_stacks==0: 
                best=steps; 
                if bsg_verbose: 
                    print(best, gclay.stacks) 
                    print(gclay.bsg_moves)
            if steps < 1000 and steps_old < steps and lb_steps < steps:
                SS.append((steps,clay))
        SS.sort(key=lambda x: x[0])
        #print(SS)
        S.clear(); i=0
        for s in SS:
            if i==w: break
            i+=1
            if s[1].steps < best:
                S.append(s[1])     
        #print(len(S))
    return best

In [6]:
def generate_candidates(lay):
    steps = lay.steps
    C = []
    for i in range(len(lay.stacks)):
        clay = copy.deepcopy(lay)
        success, clay.rank = SDpp(clay, i)
        clay.last_sd = i
        #if success != True: continue
        #success = SFpp(clay,i, rank)
        if success and clay.steps > steps:
            clay.bsg_moves.append('SDpp'+str(i))
            C.append(clay)
            
    for i in range(len(lay.stacks)):
        clay = copy.deepcopy(lay)
        success, __ = Greedy.SD_move(clay, stack=i)
        clay.last_sd = None
        #if success != True: continue
        #success = SFpp(clay,i, rank)
        if success and clay.steps > steps:
            C.append(clay)
            
    clay = copy.deepcopy(lay) 
    clay.rank = None
    clay.last_sd = None
    Greedy.lazy_greedy(clay)
    if clay.steps > steps:
        clay.bsg_moves.append('LG')
        C.append(clay)
        
    clay = copy.deepcopy(lay) 
    if clay.last_sd != None:
        success = SFpp(clay, clay.last_sd, clay.rank)
        if success and clay.steps > steps:
            clay.bsg_moves.append('SFpp'+str(clay.last_sd))
            clay.last_sd=None
            clay.rank=None
            C.append(clay)
        

        
    return C

### Solve all the instances

In [7]:
import sys

def progressbar(it, prefix="", size=60, file=sys.stdout, out=['None']):
    count = len(it)
    def show(j):
        x = int(size*j/count)
        file.write("%s[%s%s] %s\r" % (prefix, "#"*x, "."*(size-x), out[0]))
        file.flush()        
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    #file.write("\n")
    file.write(prefix + out[0] + " "*2*size+"\n")
    file.flush()

import time

for i in progressbar(range(15), "Computing: ", 40):
    time.sleep(0.1) # any calculation you need

for i in progressbar(range(15), "Computing: ", 40):
    time.sleep(0.1) # any calculation you need

Computing: None                                                                                
Computing: None                                                                                


In [12]:
import os
import copy
import numpy as np
import Greedy as gr
import Greedy
import Pixie as px
import Layout
import time
import importlib
importlib.reload(gr)
importlib.reload(Layout)
importlib.reload(px)
from os import listdir
from os.path import isfile, join


onlyfiles = [f for f in listdir("Instancias/CVS") if not isfile(join("Instancias/CVS", f))]
verbose = False
bsg_verbose=False

tot_moves=0
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0; count = 0
    av = ["None"]
    start = time.time()
    for i in progressbar(range(1,41), "CVS-"+f+ ":\t", 40, out=av):
        filename = "Instancias/CVS/"+ f+ "/data"+f+"-"+str(i)+".dat"
        layout = px.read(filename)
        steps = solve(layout)
        #steps = BSG(layout, w=20)
        #print(filename, steps)
        if steps < 1000:
            count += 1
            nmoves += steps
            av[0] = str(nmoves/count)+"\t"+str(time.time() - start) +" "*10
        else: print(filename+"\n")
    tot_moves += nmoves
    #print (f,nmoves/ count,count, " "*50);
    sys.exit
print("prom CVS:",tot_moves/(21.0*40.0))

verbose=False
dir = "Instancias\\BF"
tot_moves=0
carpetas = os.listdir(dir)
#av = ["None"]
for i in range(1,33):
    carpeta = "BF"+str(i)
    archivos = os.listdir(dir+"\\"+carpeta)
    suma = 0
    count = 0
    start = time.time()
    for archivo in archivos:
        if "optimo" not in archivo: 
            ruta = dir+"\\"+carpeta+"\\"+archivo
            #print(ruta)
            layout = px.readBF(ruta)
            steps = solve(layout)
            #steps = BSG(layout, w=20)
            if steps < 1000:
                suma += steps
                count += 1
                #av[0] = str(nmoves/count)+" "+str(time.time() - start) +" "*10
            #suma += gr.greedy_solve(layout)
            
        
    promedio = suma/count  
    tot_moves += promedio
    print(carpeta + ":\t"+str(promedio)+"\t"+str(time.time() - start))
print("prom BF:",tot_moves/(32.0))

CVS-10-10:	163.575	0.5369207859039307                                                                                          
CVS-10-6:	126.7	0.3899562358856201                                                                                          
CVS-3-3:	11.775	0.342085599899292                                                                                          
CVS-3-4:	11.875	0.3749980926513672                                                                                          
CVS-3-5:	12.575	0.3869614601135254                                                                                          
CVS-3-6:	13.375	0.39995574951171875                                                                                          
CVS-3-7:	14.975	0.4248642921447754                                                                                          
CVS-3-8:	15.5	0.40990209579467773                                                                                         

In [13]:
layout.stacks=[[26, 20, 25, 11, 19, 2, 29, 58, 35, 22, 51], [32, 16], [3, 57, 1, 44, 39, 23, 37, 33, 40, 52, 55, 53], [6, 54, 36, 7, 10, 48, 45, 34, 41, 13, 17], [38, 12, 18, 31, 15, 46, 56, 42, 59, 8, 9, 28], [24, 14, 50, 27, 47, 49, 60, 21, 4, 43, 5, 30]]
from tkinter import * 
import tkinter

def op():
    labelframe=LabelFrame(top, text="")
    labelframe1=labelframe
    return labelframe

#/////////////////////////////////////////    
top = Tk()  
#top.geometry("800x500")  
#Se obtiene tamaño de la pantalla
windowWidth = top.winfo_reqwidth()
windowHeight = top.winfo_reqheight()
positionRight = int(top.winfo_screenwidth()/4 - windowWidth/4)
positionDown = int(top.winfo_screenheight()/4 - windowHeight/4)

top.geometry("+{}+{}".format(positionRight, positionDown))
#Se crean los panels

#columnas ahora con colores
labelframe1 = LabelFrame(top, text="")
labelframe1.grid(padx=10, pady=10,columnspan=len(layout.stacks), rowspan=layout.H)


for i in range(len(layout.stacks)):
    #se obtiene el stack
    s=layout.stacks[i]
    #color
    aux=24500
    cont=1
    for item in s:
        if item <= aux:
            aux=item
            cont+=1
        else:
            break
    cont+=-1
    #Color de la columna
    color=len(s)-cont
    #/////////
    height=layout.H-len(s)
    #print(color)
    j=0
    #Aqui se imprimen los cuadros vacios para que no quede al reves el visual
    while j < height:
        tkinter.Label(labelframe1, text="",borderwidth=10, bg="white", padx=9, pady=1  ).grid(row=j,column=i)
        j+=1
    
    #Se imprimen en pantalla los valores
    for item in reversed(s):
        if color>0:
            if item > 9:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10,pady=10,bg="salmon").grid(row=j,column=i)
            else:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="salmon").grid(row=j,column=i)
            color+=-1
        else:
            if item>9:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10, pady=10,bg="lime" ).grid(row=j,column=i)
            else:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="lime" ).grid(row=j,column=i)
        #veamos si se puede pintar
        j+=1


#layout.H tiene la altura
#layout.moves tiene los movimientos
#layout.stacks tiene los stacks


#BOTONES
labelframe2 = LabelFrame(top, text="")
labelframe2.grid(padx=20, pady=20,columnspan=len(layout.stacks), rowspan=1)

button = tkinter.Button(labelframe2, text="prev",padx=13, command=op)
button.grid(row=0, column=1)

button2 = tkinter.Button(labelframe2, text="next",padx=13)
button2.grid(row=0, column=6)
#/////////////////////////////////////////////////////////////////
top.mainloop()